# Matcha-chart2text-pew

Note
- Visual Encoder. Trained on pew dataset
- Have similar model but trained with statista dataset
- The very first 100 dataset of chartqa is more similar to pew dataset. It is possible that some of the latter part is statista (need to see into it)

In [1]:
from transformers import Pix2StructProcessor, Pix2StructForConditionalGeneration
import requests
from PIL import Image

c:\Users\sung2_8l7o06c\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
processor = Pix2StructProcessor.from_pretrained('google/matcha-chart2text-pew')
model = Pix2StructForConditionalGeneration.from_pretrained('google/matcha-chart2text-pew').to(0)

In [3]:
file = '/Figures/barplot/BarPlot1/Bar0.png'
path = "./autochart"
image = Image.open(path+file)

In [4]:
inputs = processor(images=image, text="What is this chart about?", return_tensors="pt").to(0)
prediction = model.generate(**inputs, max_new_tokens=512)
prediction = processor.decode(prediction[0], skip_special_tokens=True)
print(prediction)

Just 9% of the public says that part-time workers in the United States are still and likely to be working. That’s up 13 percentage points from 2007, when 9% of the public was1.<0x0A>As it turns out, part-time workers in the United States are still and likely to be working. The public is split over whether this is an all-time high or a “ 55% say20%6%6%6%6%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7%7


In [5]:
import json

references = json.load(open(path+'/Text description/all_bar_text1.json'))

In [6]:
reference = references[0]['text']
reference

'The bar graph illustrates the number of percentage of part-time workers in luxembourg from 1991 to 1995. Each bar represents the number in the unit of Part-time workers(% of total employed). The number grows from approximately 8 Part-time workers(% of total employed) in 1991 to approximately 11 Part-time workers(% of total employed) in 1995. The number in 1995 being the peak, and the lowest number is recorded in 1991. This unique observation may need some analysis of the related historical events. '

In [8]:
import nltk
from nltk.translate import bleu
from nltk.tokenize import word_tokenize

# Reference sentences (human-generated descriptions)
reference_tokens = word_tokenize(reference)

# Candidate sentences (textual descriptions generated by the visual encoder)
prediction_tokens = word_tokenize(prediction)

# Compute BLEU score
bleu_score = bleu(reference_tokens, prediction_tokens)

print("BLEU Score:", bleu_score)


BLEU Score: 6.2856630480765226e-232


c:\Users\sung2_8l7o06c\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\sung2_8l7o06c\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
c:\Users\sung2_8l7o06c\AppData\Local\Programs\Python\Python311\Lib\site-packages\nltk\translate\bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many

In [9]:
from rouge_score import rouge_scorer
scorer = rouge_scorer.RougeScorer(['rouge1'])
scorer.score(reference, prediction)

{'rouge1': Score(precision=0.08273381294964029, recall=0.27058823529411763, fmeasure=0.12672176308539945)}

In [10]:
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

embed = hub.load("https://tfhub.dev/google/universal-sentence-encoder-large/5")

prediction_embeddings = embed([prediction])["outputs"]
reference_embeddings = embed([reference])["outputs"]

avg_prediction_embedding = np.mean(prediction_embeddings, axis=0)
avg_reference_embedding = np.mean(reference_embeddings, axis=0)

similarity_score = cosine_similarity([avg_prediction_embedding], [avg_reference_embedding])[0][0]

print("Similarity Score:", similarity_score)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

model = SentenceTransformer('bert-base-nli-mean-tokens')

prediction_embeddings = model.encode(prediction)
reference_embeddings = model.encode(reference)

avg_prediction_embedding = np.mean(prediction_embeddings, axis=0)
avg_reference_embedding = np.mean(reference_embeddings, axis=0)

similarity_score = cosine_similarity([avg_prediction_embedding], [avg_reference_embedding])[0][0]

print("Similarity Score:", similarity_score)

## For multiple evals

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import json

scores = []
references = json.load(open(path+'/Text description/all_bar_text1.json'))

for number in range(1000):
    file = f'/Figures/barplot/BarPlot1/Bar{number}.png'
    path = "./autochart"
    image = Image.open(path+file)

    inputs = processor(images=image, text="What is this chart about?", return_tensors="pt").to(0)
    prediction = model.generate(**inputs, max_new_tokens=512)
    prediction = processor.decode(prediction[0], skip_special_tokens=True)

    reference = references[number]['text']

    model = SentenceTransformer('bert-base-nli-mean-tokens')

    prediction_embeddings = model.encode(prediction)
    reference_embeddings = model.encode(reference)

    avg_prediction_embedding = np.mean(prediction_embeddings, axis=0)
    avg_reference_embedding = np.mean(reference_embeddings, axis=0)

    similarity_score = cosine_similarity([avg_prediction_embedding], [avg_reference_embedding])[0][0]

    scores.append(similarity_score)

In [ ]:
print(np.mean(scores))